In [13]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

In [3]:
#Import data frame with standardized revenues and expenditures per school district
std_districts_FL = pd.read_csv(r"..\Data\Florida data\District Finances\FL_District_Finances.csv", index_col=0)

std_districts_FL.head()

,NAME,YRDATA,TOTALREV,TFEDREV,TSTREV,TLOCREV,TOTALEXP,TCURINST,TCURSSVC,TCURONON,TCAPOUT
0,ALACHUA COUNTY SCHOOL DISTRICT,2016,9.702167,1.270125,4.333970,4.098072,9.340386,4.968435,3.352738,0.626344,0.328715
1,BAKER COUNTY SCHOOL DISTRICT,2016,8.530388,0.965414,6.065374,1.499600,8.362055,4.367453,3.266693,0.565974,0.145542
2,BAY COUNTY SCHOOL DISTRICT,2016,9.979338,0.934416,3.981534,5.063389,9.527591,5.411936,2.667003,0.541305,0.797883
3,BRADFORD COUNTY SCHOOL DISTRICT,2016,10.011229,1.574860,5.697130,2.739239,9.352776,5.072052,3.243606,0.945415,0.083593
4,BREVARD COUNTY SCHOOL DISTRICT,2016,9.968929,1.037302,4.560299,4.371328,9.390130,5.292529,2.829184,0.455725,0.526518


In [4]:
#Import data frame with crime rates per county 
FL_juv_crime_rate = pd.read_csv(r"..\Data\Florida data\Juvenile Crime by County\FL_Juvenile_crime_rate.csv", index_col=0)
FL_juv_crime_rate.head()

,Year,COUNTY NAME,TOTAL ARRESTS,JUVENILE POP,CRIME RATE
0,1998,Alachua,1854,44683,4.149229
1,1998,Baker,114,6129,1.860010
2,1998,Bay,1898,36257,5.234851
3,1998,Bradford,62,5861,1.057840
4,1998,Brevard,4721,104005,4.539205


In [5]:
#Have both data set with the same range of years: 1998-2016
FL_juv_crime_rate = FL_juv_crime_rate.loc[FL_juv_crime_rate['Year'] != 2017]


In [6]:
#Pull out county names from the school districts
std_districts_FL['COUNTY NAME'] = std_districts_FL.NAME.str.title().str.split().str[0]


In [7]:
#This loop deals with the few exceptions that weren't covered in the previous command

for i, row in std_districts_FL.iterrows():
    for x in ["ST JOHNS", "ST LUCIE", "SANTA ROSA", "DE SOTO", "INDIAN RIVER"]:
        if x in std_districts_FL['NAME'][i]:
            std_districts_FL.at[i,'COUNTY NAME'] = x   
        

In [8]:
std_districts_FL.rename(columns={'YRDATA': 'Year'}, inplace=True)
juv_crime_vs_finances = std_districts_FL.merge(FL_juv_crime_rate, on=["COUNTY NAME","Year"], how = 'inner')

list(juv_crime_vs_finances)

['NAME',
 'Year',
 'TOTALREV',
 'TFEDREV',
 'TSTREV',
 'TLOCREV',
 'TOTALEXP',
 'TCURINST',
 'TCURSSVC',
 'TCURONON',
 'TCAPOUT',
 'COUNTY NAME',
 'TOTAL ARRESTS',
 'JUVENILE POP',
 'CRIME RATE']

In [9]:
cols = ['COUNTY NAME',
     'Year',
     'TOTALREV',
     'TFEDREV',
     'TSTREV',
     'TLOCREV',
     'TOTALEXP',
     'TCURINST',
     'TCURSSVC',
     'TCURONON',
     'TCAPOUT',
     'TOTAL ARRESTS',
     'JUVENILE POP',
     'CRIME RATE']

juv_crime_vs_finances = juv_crime_vs_finances[cols]

In [10]:
juv_crime_vs_finances.head()

,COUNTY NAME,Year,TOTALREV,TFEDREV,TSTREV,TLOCREV,TOTALEXP,TCURINST,TCURSSVC,TCURONON,TCAPOUT,TOTAL ARRESTS,JUVENILE POP,CRIME RATE
0,Alachua,2016,9.702167,1.270125,4.333970,4.098072,9.340386,4.968435,3.352738,0.626344,0.328715,730,47667,1.531458
1,Baker,2016,8.530388,0.965414,6.065374,1.499600,8.362055,4.367453,3.266693,0.565974,0.145542,77,6877,1.119674
2,Bay,2016,9.979338,0.934416,3.981534,5.063389,9.527591,5.411936,2.667003,0.541305,0.797883,1234,39597,3.116398
3,Bradford,2016,10.011229,1.574860,5.697130,2.739239,9.352776,5.072052,3.243606,0.945415,0.083593,60,5393,1.112553
4,Brevard,2016,9.968929,1.037302,4.560299,4.371328,9.390130,5.292529,2.829184,0.455725,0.526518,2250,107226,2.098372


In [11]:
juv_crime_vs_finances.to_csv(r"..\Data\Florida data\Merged_Datasets\juvenile_crime_vs_finances.csv")

In [19]:
#Linear model without a constant

cols_model = ['TFEDREV',
     'TSTREV',
     'TLOCREV',
     'TCURINST',
     'TCURSSVC',
     'TCURONON',
     'TCAPOUT']

X = juv_crime_vs_finances[cols_model]
y = juv_crime_vs_finances["CRIME RATE"]

# Note the difference in argument order
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()



<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             CRIME RATE   R-squared:                       0.715
Model:                            OLS   Adj. R-squared:                  0.713
Method:                 Least Squares   F-statistic:                     404.5
Date:                Sun, 18 Nov 2018   Prob (F-statistic):          1.22e-302
Time:                        15:57:06   Log-Likelihood:                -2124.1
No. Observations:                1138   AIC:                             4262.
Df Residuals:                    1131   BIC:                             4297.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
TFEDREV       -0.9032      0.152     -5.956      0.000      -1.201      -0.606
TSTREV         0.3845      0.055      6.985      0.000       0.277       0.493
TLOCREV        0.3906      0.056      6.996      0.000       0.281       0.500
TCURINST       0.0342      0.091      0.377      0.706      -0.144       0.212
TCURSSVC       0.1818      0.134      1.359      0.174      -0.081       0.444
TCURONON      -0.3294      0.261     -1.264      0.206      -0.841       0.182
TCAPOUT       -0.1570      0.047     -3.326      0.001      -0.250      -0.064
==============================================================================
Omnibus:                      332.188   Durbin-Watson:                   1.574
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1915.609
Skew:                           1.219   Prob(JB):                         0.00
Kurtosis:                       8.870   Cond. No.                         45.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [22]:


X = juv_crime_vs_finances['TOTALEXP']

y = juv_crime_vs_finances["CRIME RATE"]

# Note the difference in argument order
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             CRIME RATE   R-squared:                       0.664
Model:                            OLS   Adj. R-squared:                  0.664
Method:                 Least Squares   F-statistic:                     2245.
Date:                Sun, 18 Nov 2018   Prob (F-statistic):          2.00e-271
Time:                        15:58:17   Log-Likelihood:                -2217.2
No. Observations:                1138   AIC:                             4436.
Df Residuals:                    1137   BIC:                             4441.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
TOTALEXP       0.2515      0.005     47.385      0.000       0.241       0.262
==============================================================================
Omnibus:                      313.811   Durbin-Watson:                   1.356
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1736.673
Skew:                           1.156   Prob(JB):                         0.00
Kurtosis:                       8.593   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""